# Reproduction of Deepmind’s StarCraft II Research by Using Higher-Level Framework

Author: Shengyi (Costa) Huang, Costa.Huang@outlook.com

## Introduction

In early 2017, Google Deepmind introduced a python library [``Pysc2``](https://github.com/deepmind/pysc2), the SC2LE (StarCraft II Learning Environment). It provides a interface for RL (Reinforcement Learning) agents to interact with StarCraft 2 by providing the observations and receiving actions. In their paper, Deepmind also described some baseline training algorithms and used them to train agents in [mini-games](https://github.com/deepmind/pysc2/blob/master/docs/mini_games.md) such as ``BuildMarines``, ``DefeatRoaches`` and ``MoveToBeacon``<cite data-cite="Vinyals2017-ck"> (Vinyals, 2017)</cite>. Nevertheless, the implementation of the algorithm is not revealed. In this research project, we surveys some reproduction of Deepmind's result and, in the end, we reproduces the results by using higher-level framework (e.g. Tensorforce and gym).

## Basic Mechanics of Pysc2

One of the most helpful tutorial is from [Building a Basic PySC2 Agent](https://chatbotslife.com/building-a-basic-pysc2-agent-b109cde1477c) and [Building a Smart PySC2 Agent](https://chatbotslife.com/building-a-smart-pysc2-agent-cdc269cb095d). In the simplest form, you create a class that inherits from ``base_agent.BaseAgent`` and override the ``step()`` function. In essence, the ``step`` function of ``base_agent.BaseAgent`` gives you the observation of current state, including units killed, rewards, and etc, and you need to return an action.

In [12]:
from pysc2.agents import base_agent
from pysc2.lib import actions

class SimpleAgent(base_agent.BaseAgent):
    def step(self, obs):
        super(SimpleAgent, self).step(obs)
        
        return actions.FunctionCall(actions.FUNCTIONS.no_op.id, [])

Suppose you follow the author's instruction and create a [``simple_agent.py``](https://github.com/skjb/pysc2-tutorial/blob/master/Building%20a%20Basic%20Agent/simple_agent.py) and run
```cmd
python -m pysc2.bin.agent \
--map Simple64 \
--agent simple_agent.SimpleAgent \
--agent_race T
```

## Existing Reproduction

Some of the popular (most starred) reproduction repos are:
* [pysc2-examples](https://github.com/chris-chris/pysc2-examples)
* [pysc2-agents](https://github.com/xhujoy/pysc2-agents)
* [sc2aibot](https://github.com/pekaalto/sc2aibot)
* [pysc2-RLagents](https://github.com/greentfrapp/pysc2-RLagents)


In [10]:
%%HTML
<style> 
code {
    background-color : #eff0f1 !important;
    padding: 1px 5px !important;
}

</style>